In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

def generate_random_number(number,x_data,y_data):
    num_samples = number
    data_numb=np.zeros((num_samples, 28, 100, 1), dtype = np.float32)
    data_labe=np.empty((num_samples, 1), dtype = np.chararray)
    for i in range(num_samples):
        digit1 = np.random.choice(y_data)
        digit2 = np.random.choice(y_data)
        digit3 = np.random.choice(y_data)
        digit4 = np.random.choice(y_data)
        digit5 = np.random.choice(y_data)
        
        length = np.random.randint(1, 6)
        canvas = np.zeros((28, 100, 1), dtype=np.float32)
        labe = np.zeros(length,dtype=np.int_)
        
        lent_front = 0
        for n in range(length):
            digit_labe =[digit1, digit2, digit3, digit4, digit5]
            digit = digit_labe[n]
            id=np.random.choice(np.where(y_train == digit)[0])
            if n==0:
                lent_front = 28*n
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            else:
                numb=np.random.randint(8,18)
                lent_front = lent_front+numb
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            canvas_add = np.concatenate([x_data_front, x_data[id]], 1)
            canvas_add = np.concatenate([canvas_add, x_data_after],  1)
            canvas += canvas_add
            labe[n] = y_data[id]
            
        datalabe = [str(x) for x in labe]
        data_labe[i] = ''.join(datalabe)
        data_numb[i]=canvas 
    print('end')
    return data_numb, data_labe

In [2]:

def get_label(y_lab):
    max_length = max(len(sublist[0]) for sublist in y_lab)
    lab_list = [[int(char) if len(char) == 1 else 11 for char in sublist[0]] + [10] * (max_length - len(sublist[0])) for sublist in y_lab]
    lab = np.array(lab_list)
    return lab

In [3]:

x_data_random , y_data_random = generate_random_number(100000,x_train, y_train)
y_data_ran_ctc = get_label(y_data_random)

end


In [4]:
from tensorflow.keras.utils import to_categorical
y_data_ran = to_categorical(y_data_ran_ctc, num_classes=11)

In [5]:
x_train_ran , y_train_ran, y_train_ran_ctc, y_train_random = x_data_random[:60000] , y_data_ran[:60000] , y_data_ran_ctc[:60000] , y_data_random[:60000]
x_test_ran , y_test_ran, y_test_ran_ctc, y_test_random= x_data_random[60000:80000] , y_data_ran[60000:80000] , y_data_ran_ctc[60000:80000] , y_data_random[60000:80000]
x_verify_ran , y_verify_ran, y_verify_ran_ctc, y_verify_random = x_data_random[80000:] , y_data_ran[80000:] , y_data_ran_ctc[80000:] , y_data_random[80000:]

In [6]:
from PIL import Image
from IPython.display import display

for id_n in np.random.randint(0, len(y_train_random),10):
    image_data = (np.squeeze(x_train_ran[id_n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    
    title = f"Label:{y_train_random[id_n][0]}"
    print(title)
    display(image)

Label:48187


Label:4


Label:36128


Label:70


Label:28


Label:1756


Label:71141


Label:756


Label:4


Label:11


In [7]:
max_str_len = 5 # max length of input labels
num_of_characters = 11 # +1 for ctc pseudo blank
num_of_timestamps = 64 # max length of predicted labels

In [8]:
train_label_len = np.zeros([len(y_train_random), 1])
train_input_len = np.ones([len(y_train_random), 1]) * num_of_timestamps
train_output = np.zeros([len(y_train_random)])

for i in range(len(y_train_random)):
    train_label_len[i] = len(y_train_random[i][0]) 

print('True label : ',y_train_random[100] , '\ntrain_y : ',y_train_ran_ctc[100],'\ntrain_label_len : ',train_label_len [100], 
      '\ntrain_input_len : ', train_input_len[100])

True label :  ['23390'] 
train_y :  [2 3 3 9 0] 
train_label_len :  [5.] 
train_input_len :  [64.]


In [9]:
test_label_len = np.zeros([len(y_test_random), 1])
#test_input_len = np.ones([len(y_test_random), 1]) * (num_of_timestamps-2)
test_input_len = np.ones([len(y_test_random), 1]) * num_of_timestamps
test_output = np.zeros([len(y_test_random)])

for i in range(len(y_test_random)):
    test_label_len[i] = len(y_test_random[i][0]) 

print('True label : ',y_test_random[10] , '\ntest_y : ',y_test_ran_ctc[10],'\ntest_label_len : ',test_label_len [10], 
      '\ntest_input_len : ', test_input_len[10])

True label :  ['4'] 
test_y :  [ 4 10 10 10 10] 
test_label_len :  [1.] 
test_input_len :  [64.]


In [10]:
verify_label_len = np.zeros([len(y_verify_random), 1])
verify_input_len = np.ones([len(y_verify_random), 1]) * num_of_timestamps
verify_output = np.zeros([len(y_verify_random)])

for i in range(len(y_verify_random)):
    verify_label_len[i] = len(y_verify_random[i][0]) 

print('True label : ',y_verify_random[1] , '\nverify_y : ',y_verify_ran_ctc[1],'\nverify_label_len : ',verify_label_len [1], 
      '\nverify_input_len : ', verify_input_len[1])

True label :  ['92011'] 
verify_y :  [9 2 0 1 1] 
verify_label_len :  [5.] 
verify_input_len :  [64.]


In [13]:
import tensorflow as tf

from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, GRU, Dense, Lambda, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

def ConvBlock(layout,filters):
        inner = Conv2D(filters, (3, 3), activation='elu', padding='same')(layout)  
        inner = Activation('elu')(inner)
        layout = BatchNormalization()(inner)
        return layout

input_data = Input(shape=(28, 100, 1), name='input')

inner = ConvBlock(input_data,16) 
inner = ConvBlock(inner,32) 
inner = MaxPooling2D(pool_size=(1, 2), name='max1')(inner)
inner = ConvBlock(inner,32) 
inner = ConvBlock(inner,64) 
inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)
inner = ConvBlock(inner,64) 
inner = ConvBlock(inner,128) 
inner = MaxPooling2D(pool_size=(1, 2), name='max4')(inner)
inner = ConvBlock(inner,128)
inner = ConvBlock(inner,256)  
inner = MaxPooling2D(pool_size=(1, 2), name='max2')(inner)
inner = Dropout(0.2)(inner)

# CNN to RNN
inner = Reshape(target_shape=((64,28*24)), name='reshape')(inner)
inner = Dense(64, activation="softmax", name='dense1', kernel_regularizer = regularizers.l2(0.01))(inner)#

## RNN
inner = Bidirectional(GRU(256, return_sequences=True, dropout=0.2), name = 'lstm1')(inner)
inner = Bidirectional(GRU(512, return_sequences=True, dropout=0.2), name = 'lstm2')(inner)

## OUTPUT
inner = Dense(num_of_characters, kernel_regularizer = regularizers.l2(0.01),name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)

model = Model(inputs=input_data, outputs=y_pred)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 100, 1)]      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 28, 100, 16)       160       
_________________________________________________________________
activation_16 (Activation)   (None, 28, 100, 16)       0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 28, 100, 16)       64        
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 28, 100, 32)       4640      
_________________________________________________________________
activation_17 (Activation)   (None, 28, 100, 32)       0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 28, 100, 32)       128   

In [14]:
# the ctc loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage
    #y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [15]:

labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

In [15]:
# the loss calculation occurs elsewhere, so we use a dummy lambda function for the loss
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau 
model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])


history = model_final.fit(x=[x_train_ran, y_train_ran_ctc, train_input_len, train_label_len], y=train_output, 
                validation_data=([x_test_ran, y_test_ran_ctc, test_input_len, test_label_len], test_output),
                epochs=100, verbose=1, batch_size=16)

Epoch 1/100
3750/3750 [==============================] - 241s 62ms/step - loss: 9.0745 - accuracy: 0.0000e+00 - val_loss: 6.6030 - val_accuracy: 0.0000e+00
Epoch 2/100
3750/3750 [==============================] - 233s 62ms/step - loss: 5.8336 - accuracy: 0.0278 - val_loss: 4.6103 - val_accuracy: 0.0975
Epoch 3/100
3750/3750 [==============================] - 233s 62ms/step - loss: 4.2682 - accuracy: 0.1396 - val_loss: 3.5682 - val_accuracy: 0.1949
Epoch 4/100
3750/3750 [==============================] - 234s 62ms/step - loss: 3.5038 - accuracy: 0.2012 - val_loss: 2.9845 - val_accuracy: 0.2299
Epoch 5/100
3750/3750 [==============================] - 231s 62ms/step - loss: 3.0071 - accuracy: 0.2451 - val_loss: 2.6462 - val_accuracy: 0.2907
Epoch 6/100
3750/3750 [==============================] - 234s 62ms/step - loss: 2.6408 - accuracy: 0.3011 - val_loss: 2.3561 - val_accuracy: 0.3344
Epoch 7/100
3750/3750 [==============================] - 234s 63ms/step - loss: 2.3289 - accuracy: 0.354

In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Model accuracy", "Model loss"))

# Add the first subplot for accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                         y=history.history['accuracy'], mode='lines',
                         name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                         y=history.history['val_accuracy'], mode='lines',
                         name='Validation', line=dict(color='orange')), row=1, col=1)

# Add the second subplot for loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'], mode='lines',
                         name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'], mode='lines',
                         name='Validation', line=dict(color='orange'), showlegend=False), row=1, col=2)

fig.update_layout(title="Model Training History",
                  xaxis_title="Epoch",
                  yaxis_title="Value",
                  legend=dict(x=0.85, y=1))

fig.show()

In [ ]:
# the loss calculation occurs elsewhere, so we use a dummy lambda function for the loss
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau 
model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])


history = model_final.fit(x=[x_train_ran, y_train_ran_ctc, train_input_len, train_label_len], y=train_output, 
                validation_data=([x_test_ran, y_test_ran_ctc, test_input_len, test_label_len], test_output),
                epochs=150, verbose=1, batch_size=16)

Epoch 1/120
3750/3750 [==============================] - 274s 71ms/step - loss: 7.8393 - accuracy: 0.0450 - val_loss: 4.5982 - val_accuracy: 0.1654
Epoch 2/120
3750/3750 [==============================] - 266s 71ms/step - loss: 3.7549 - accuracy: 0.2201 - val_loss: 2.8519 - val_accuracy: 0.3059
Epoch 3/120
3750/3750 [==============================] - 264s 71ms/step - loss: 2.6572 - accuracy: 0.3325 - val_loss: 2.2082 - val_accuracy: 0.3812
Epoch 4/120
3750/3750 [==============================] - 264s 70ms/step - loss: 2.0251 - accuracy: 0.4218 - val_loss: 1.6993 - val_accuracy: 0.4943
Epoch 5/120
3750/3750 [==============================] - 264s 71ms/step - loss: 1.6100 - accuracy: 0.4988 - val_loss: 1.3827 - val_accuracy: 0.5587
Epoch 6/120
3750/3750 [==============================] - 264s 70ms/step - loss: 1.3284 - accuracy: 0.5622 - val_loss: 1.2379 - val_accuracy: 0.6026
Epoch 7/120
3750/3750 [==============================] - 264s 70ms/step - loss: 1.1285 - accuracy: 0.6183 - val_

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Model accuracy", "Model loss"))

# Add the first subplot for accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                         y=history.history['accuracy'], mode='lines',
                         name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                         y=history.history['val_accuracy'], mode='lines',
                         name='Validation', line=dict(color='orange')), row=1, col=1)

# Add the second subplot for loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'], mode='lines',
                         name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'], mode='lines',
                         name='Validation', line=dict(color='orange'), showlegend=False), row=1, col=2)

fig.update_layout(title="Model Training History",
                  xaxis_title="Epoch",
                  yaxis_title="Value",
                  legend=dict(x=0.85, y=1))

fig.show()

In [20]:
preds = model.predict(x_verify_ran)
decoded = K.get_value(K.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1], 
                                   greedy=True)[0][0])

alphabets = u"0123456789 "

def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # CTC Blank
            break
        else:
            ret+=alphabets[ch]
    return ret

prediction = []
for i in range(len(y_verify_ran)):
    prediction.append(num_to_label(decoded[i]))

In [21]:
y_true = []
for i in range(20000):
    y_true.append(y_verify_random[i][0])

In [22]:
correct_char = 0
total_char = 0
correct = 0
valid_size=len(y_verify_random)

for i in range(valid_size):
    pr = prediction[i]
    tr = y_true[i]
    total_char += len(tr)
    
    for j in range(min(len(tr), len(pr))):
        if tr[j] == pr[j]:
            correct_char += 1
            
    if pr == tr :
        correct += 1 
    
print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))
print('Correct words predicted      : %.2f%%' %(correct*100/valid_size))

Correct characters predicted : 96.26%
Correct words predicted      : 89.91%


In [23]:
for id_n in np.random.randint(0, len(y_true),10):
    image_data = (np.squeeze(x_verify_ran[id_n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    title = f"Label:{y_true[id_n]}; Predicted:{prediction[id_n]}"
    print(title)
    display(image)

Label:7; Predicted:7


Label:88; Predicted:88


Label:797; Predicted:797


Label:86880; Predicted:86880


Label:33617; Predicted:33617


Label:2; Predicted:2


Label:259; Predicted:259


Label:36; Predicted:36


Label:3; Predicted:3


Label:0; Predicted:0
